# Abell 1656: the Coma Cluster of Galaxies

Stefania Amodeo¹, Thomas Boch¹, Caroline Bot¹, Giulia Iafrate², Katharina A. Lutz¹, Manon Marchand¹, Massimo Ramella², and Jenny G.Sorce¹

1. Université de Strasbourg, CNRS, Observatoire Astronomique de Strasbourg, UMR 7550, F-67000, Strasbourg, France
2. INAF - Osservatorio Astronomico di Trieste

The original form of this tutorial authored by Massimo Ramella & Giulia Iafrate can be found on the [EURO-VO tutorials page](http://www.euro-vo.org/?q=science/scientific-tutorials). The version here is an adaptation into a jupyter notebook by the Strasbourg astronomical Data Center (CDS) team. 

***

## Introduction

The Coma Cluster is an ensemble of over 1000 galaxies that takes its name from the constellation Coma Berenices. 

The goals of this notebook tutorial are to:
 1. examine the Coma cluster of galaxies (Abell 1656) using services and data from the virtual observatory within a jupyter notebook. This allows a quick evaluation of the mean redshift and velocity dispersion of the cluster. Both measurements are important to study the evolution of galaxy clusters. To do so we use redshifts and photometry (Petrosian r magnitude) from the Sloan Digital Sky Survey ([SDSS](https://www.sdss.org/)) and then add redshifts from the Cluster And Infall Region Nearby Survey ([CAIRNS](https://iopscience.iop.org/article/10.1086/378599)) (Rines et al. 2003). This improves the completeness of the redshift sample,
 2. research the Mikulski Archive for Space Telescopes ([MAST](https://archive.stsci.edu/)) for Hubble Space Telescope (HST) spectra in the region of the Coma cluster,
 3. download a spectrum from MAST and do a quick analysis of the redshift of the emission lines in the spectrum. 

In [4]:
# general python packages
import numpy as np
from scipy.optimize import curve_fit

# for visualisation
import ipyaladin.aladin_widget as ipyal
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# to query astronomical databases
from astroquery.vizier import Vizier
from astroquery.xmatch import XMatch
from astroquery.simbad import Simbad
import pyvo

# libraries for handling astronomy data
from astropy import units as u
from astropy.io import fits
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord
from specutils import Spectrum1D, SpectralRegion
import specutils.analysis as spec_ana

print('Ready to go :) ')


Ready to go :) 


## Step #1: Exploration of the Coma cluster

### Display the region of Abell 1656 in Aladin lite

We start by displaying the Coma Cluster in an Aladin Lite widget. We will display DSS2 color images (`survey='P/DSS2/color'`) centered on Abell 1656 (`target='A1656'`) and set the field of view to $0.7^\circ$ (`fov=0.7`). 
At this distance, this field of view corresponds to approximately 1Mpc: this is large enough to look at the cluster. 

If you are using Jupyter lab you can open a second python3 notebook. On this new notebook click the "Python 3" button in the top-right corner of this notebook to switch kernel. A new window will pop up and you can then select this notebook (i.e. "Abel1656...") as a kernel. This will link the two notebooks such that they see the same variables. You may use the second notebook to look at Aladin Lite widget while doing the analysis in this notebook. Please see the ipyaladin webpage for more information on how to use the widget in Jupyter lab. 

In [ ]:
aladin = ipyal.Aladin(target='A1656', fov=0.7, survey='P/DSS2/color')
aladin


You can already see galaxies composing the cluster, and a few stars, like for example the bright [HD112887 star](https://simbad.cds.unistra.fr/simbad/sim-coo?Coord=12+59+32.92000000000+%2B28+03+56.7300000000&Radius=10&Radius.unit=arcmin&submit=submit+query) (click for a surprise Simbad query). But there is more! As with any Aladin Lite implementation, you can interact with this widget. 
 
 - to zoom in and out, place your mouse pointer on the image and scroll,
 - you can right-click on a point to grab it and center the reticule on it, 
 - with <img src="images/ipyaladin_layer.png" alt="the Layer Button" style="width:30px; display: inline-block;"/>  you can select other image surveys,
 - if you would like to look at another target, you can use the search field <img src="images/ipyaladin_search.png" alt="the Search Button" style="width:30px; display: inline-block;"/> to get there. 

We can also interact with the variable `aladin`. If, for example, after zooming in and out, you wanted to set the FoV again to $0.7^\circ$, run:

In [ ]:
aladin.fov = 0.7


and look back up ... or call the variable `aladin` again to pop a new widget window. You'll remark that the field of view and reticule position are synchronized, but not the survey. It can allow the exploration of the same region in two different wavelengths. 

In [ ]:
aladin


### Load the SDSS catalog and select galaxies

In this section, we will access the SDSS DR12 catalog among the [VizieR](https://vizier.cds.unistra.fr/) catalog service. We will download all entries that are located within 40arcmin of the center of the cluster A1656. We start by querying VizieR for all catalogs that match the term `SDSS DR12`.

Since this query takes a few seconds, we will store its results in the variable `catalog_list_sdss` for further analysis.

In [ ]:
catalog_list_sdss = Vizier.find_catalogs('SDSS DR12')
print(f'The catalog is returned and stored in a {type(catalog_list_sdss)}')


To know what is in this ordered dictionary, we look at its first entry and list the methods applicable to the associated value.

In [ ]:
# just selecting the value of the first item in the dictionnary
first_entry_value = catalog_list_sdss[next(iter(catalog_list_sdss))]
# the dir() function is built in python > 3. It lists all available methods for an object
print(dir(first_entry_value))


Among the possible methods, we print the `description`, but feel free to explore the catalog list :)

In [ ]:
# Let's display a list of the catalogs' names and descriptions
for key, value in catalog_list_sdss.items():
    print(key, ': ', value.description)


In this list, the first item is the complete SDSS catalog, while the others are sub-catalogs produced from the full one that often adds information. These other catalogs are published in astronomical journals (ex: ApJ stands for the Astrophysical Journal). 

We are interested in data from the main SDSS DR12 photometric catalog: "The SDSS Photometric Catalogue, Release 12 (Alam+, 2015)". We query its content in 10arcmin around the center of the Coma Cluster.

In [ ]:
results_test_sdss = Vizier.query_region(
    "A1656", radius="0d10m0s", catalog='V/147')
print(results_test_sdss)


As you see, there is only one table in our resulting list of tables. Moreover, this table has only 50 rows, an extremely small number! It is actually due to the default limit in the number of rows of the `Vizier.query_region` function. For now, we will use this small table to see what columns could be interesting. We will query without a row limit once we know what we need. This is a general good practice and allows to avoid unneccesary manipulation of huge tables.

To access the unique test table in the list, we select the first table of the catalogue with `results_test_sdss[0]`. And we can again print the list of available methods for this object:

In [ ]:
print(dir(results_test_sdss[0]))


Let's see its columns:

In [ ]:
print(results_test_sdss[0].colnames)


But since the names don't mean a lot by themselves, let's also look at another method, the `info` one that gives more detailed information about each column. If these descriptions are still not clear, you can also have a look directly at the [SDSS DR12 webpage](https://www.sdss.org/dr12/).

In [ ]:
print(results_test_sdss[0].info)


For our current goal, we will need the coordinates (`RA_ICRS` and `DE_ICRS`), r-band magnitudes (`rmag`),and redshifts (`zsp`) of the galaxies in the Coma cluster. Hence, we only keep these columns. 

SDSS furthermore provides information on the type of source (galaxies correspond to `class` = 3, see the `info` output above) so we'll want to keep these. 

Additionally, and this information is specific to the SDSS survey (see [their website](https://www.sdss.org/dr12/help/glossary/)), some objects are observed more than once because of the overlaps between plates. To avoid these duplicates, we select only the primary observation of each object (corresponding to `mode` = 1). 

A first way of obtaining unique records (`mode` = 1) for the galaxies (`class` = 3) would be to apply masks on the precedent test table:

In [ ]:
masked_sdss = results_test_sdss[0]
masked_sdss = masked_sdss[(masked_sdss['mode'] == 1)
                          & (masked_sdss['class'] == 3)]
# note the important double [] here
masked_sdss[['RA_ICRS', 'DE_ICRS', 'rmag', 'zsp']].pprint()


But here is a second option using a specialized instance of the Vizier class in which we will lift the restriction on the number of rows.

To know how to do so, always use the magic help in jupyter notebooks:

In [3]:
Vizier.info()


NameError: name 'Vizier' is not defined

Where we can directly read that we should give the `columns` names as a list, the `column_filters` as a dictionary, and the `row_limit` as an int. It is even specified that `row_limit` = -1 puts no limit on the number of rows. So let us do this:

In [ ]:
vizier_instance = Vizier(columns=['RA_ICRS', 'DE_ICRS', 'rmag', 'zsp'],
                         column_filters={'class': '==3', 'mode': '==1'},
                         row_limit=-1)


Now that we have prepared everything, we can query VizieR for all galaxies, which are primary sources and within 40arcmin of the center of the Coma cluster. Note that VizieR knows that `A1656` is the center of the Coma cluster. If you wanted to search at some other place in the sky, you could also give coordinates (in the form of Astropy's `SkyCoord`) instead of `'A1656'`.

In [ ]:
results_coma_sdss = vizier_instance.query_region('A1656',
                                                 radius='0d40m0s',
                                                 catalog='V/147')
print(results_coma_sdss)


As you can see, the result of our query includes data from one catalog for 23770 objects (*i.e.* the table has 23770 rows). The output from this query is again a list of Astropy Table objects. We assign the table to the variable `sdss` and work with this from here on. 

In [ ]:
sdss = results_coma_sdss[0]
sdss.show_in_notebook(display_length=15)


By navigating this table, you'll remark that very few entries have a spectroscopic redshift `zsp`. Thus we will complete the results later with information from another table. But we'll first ensure the validity of entries here. 

### Identify the brightest sources as stars contaminating the sample

We have already restricted our sample to sources that are classified as galaxies (`class` = 3). However, for very bright sources, stars might be confused with galaxies. To test this and exclude any contamination from stars, we now take a closer look at the brightest sources. To do so we select sources brighter than 11.5 mag in r-band (`rmag < 11.5`) and check with the Aladin widget what these sources look like.

In [ ]:
stars = sdss[sdss['rmag'] < 11.5]
aladin.add_table(stars['RA_ICRS', 'DE_ICRS'])
print(f'Our sample contains {len(stars)} really bright sources.')


With `aladin.add_table(stars)`, we have added symbols to the Aladin Lite widget at the location of the brightest sources (*i.e.* stars). If you now scroll back up to the Aladin Lite widget and zoom out, you will be able to find all the brightest sources. By looking at each source you will find that these are indeed stars. 

You can choose to hide or show the symbols for the table entries in the <img src="images/ipyaladin_layer.png" alt="the Layer Button" style="width:30px; display: inline-block;"/> options, the tableappears as `catalog`

In [ ]:
aladin


### Build a subset of galaxies with photometry and redshift in SDSS

Now on to exploring the galaxies in our sample. 
We again build a subset, this time for all sources fainter than 11.5mag (to leave out the stars identified in the section above) but brighter than 17.77mag, which is the [completeness limit of the SDSS spectroscopic sample](https://www.sdss.org/dr12/algorithms/legacy_target_selection/). 

In [ ]:
sdss_sample = sdss[(sdss['rmag'] > 11.5) & (sdss['rmag'] < 17.77)]
sdss_sample.show_in_notebook(display_length=15)


### Improve the completeness with other sources of redshifts in Vizier

As you can see in the table above, not all galaxies in our zsp17 sample have redshift measurements (some rows have '--' in the 'zsp' column, i.e. they are masked). So to improve the completeness of our sample we will now use Vizier to search for redshifts in the Rines et al (2003) catalog. First, find all catalogs that match the search terms 'redshifts Rines 2003':

In [ ]:
catalog_list_rines = Vizier.find_catalogs('Rines 2003')
for k, v in catalog_list_rines.items():
    print(k, ': ', v.description)


Among these, we find the 'J/AJ/126/2152' catalog:  Cluster And Infall Region Nearby Survey. I (Rines+, 2003): the one we'd like to read.

So again, let's take a quick look at this catalog. 

In [ ]:
results_test_rines = Vizier.query_region(
    "A1656", radius="0d10m0s", catalog='J/AJ/126/2152')
print(results_test_rines)


In [ ]:
results_test_rines[0]


In [ ]:
results_test_rines[1]


In [ ]:
results_test_rines[1].info()


After inspecting the result of the test query, we see that the first table describes the cluster as an ensemble. The second one describes individual galaxies in the cluster. The later, named `1:J/AJ/126/2152/galaxies` contains the information we want.

To see which of the galaxies in the `1:J/AJ/126/2152/galaxies` table could fill in the gaps in our SDSS table, we first isolate the galaxies without redshifts in `zsp17`. Then we spatially crossmatch the two tables using the [CDS XMatch service](http://cdsxmatch.u-strasbg.fr/xmatch) via the `astroquery.XMatch.query` module. 

In [ ]:
# mask zsp17 for entries with a nan value in the zsp column
sdss_sample_without_zsp = sdss_sample[np.isnan(sdss_sample['zsp'])]
sdss_sample_with_zsp = sdss_sample[~np.isnan(sdss_sample['zsp'])]
sdss_sample_without_zsp.show_in_notebook(display_length=15)


In [ ]:
xmatch_sdss_rines = XMatch.query(cat1=sdss_sample_without_zsp,
                                 cat2='vizier:J/AJ/126/2152/galaxies',
                                 max_distance=5 * u.arcsec, colRA1='RA_ICRS',
                                 colDec1='DE_ICRS')
xmatch_sdss_rines.show_in_notebook(display_length=15)


### Build the final catalog including the Rines et al. (2003) redshifts

The resulting table of the cross-match above contains 25 rows, so we have found recession velocity ('cz') measurements for 25 galaxies. Now let's add these data to the zsp17_with table.

In [ ]:
# converts redshift into velocity
c = 2.998e5  # km/s speed of light
sdss_sample_with_zsp['cz'] = sdss_sample_with_zsp['zsp'] * c
# from the cross math result, only keep columns that are in sdss_sample_with_zsp
to_complete = xmatch_sdss_rines[sdss_sample_with_zsp.colnames]
# put together the two tables
complete_sample = vstack([sdss_sample_with_zsp, to_complete])
# don't forget to set the unit of the newly created column
complete_sample['cz'].unit = (u.km / u.s)
complete_sample.pprint()


Now we have a table with all galaxies that either have a redshift measurement in SDSS or a velocity value obtained by (Rines *et al.* 2003).

Overall, this sample within 40arcmin of the center of the Coma cluster contains 514 galaxies.

Before we start the analysis of the data, let's look at the galaxies in the sample by loading the table into the Aladin Lite widget. They will appear as `catalog_1` and with a different color than the bright stars we added before. 

In [ ]:
aladin.add_table(complete_sample)


### Determine velocity distribution, cluster average velocity, and velocity dispersion

Based on the 514 galaxies, we can now analyze the recession velocity and velocity dispersion of the Abell 1656 galaxy cluster. First, we visualize the recession velocity distribution of the entire sample:

In [ ]:
DF_complete_sample = complete_sample.to_pandas()


In [ ]:
sns.set_style("darkgrid")
sns.displot(data=DF_complete_sample, x='cz', bins=30)


Note how there is a large range of recession velocities in our sample. We are only interested in the range of recession velocities of the Coma cluster. These are around the peak at low velocities. Thus, we restrict our sample to a subset `DF_Coma` to recession velocities between 3000 and 11000 km/s:

In [ ]:
DF_Coma = DF_complete_sample[(DF_complete_sample['cz'] > 3000.)
                             & (DF_complete_sample['cz'] < 11000.)]

sns.displot(data=DF_Coma, x='cz')


This subset corresponds to galaxies in the vicinity of the cluster (both spatially and in recession velocity). Let's calculate the mean recession velocity of the cluster and its velocity dispersion:

In [ ]:
print(f"""The mean velocity in Coma is {DF_Coma['cz'].mean()} km/s.
Its velocity dispersion (i.e. standard deviation) is {DF_Coma['cz'].std()} km/s.""")


It is in agreement with more refined analyses (e.g. [Sohn et al. 2017, ApJS, 229, 20](https://iopscience.iop.org/article/10.3847/1538-4365/aa653e)).

When looking back at the query results for the Rines et al. (2003) catalog, you can check again the table that describes the full cluster. The mean recession velocity `cz` = 6973km/s and dispersion `sigmap_3s_` = 1042km/s for the Coma cluster are is also in good agreement with our results. 

In [ ]:
results_test_rines[0]


## Search for Hubble Space Telescope (HST) spectra from the Coma Cluster

We now want to find out whether there are HST spectra available for the galaxies that had neither a redshift in SDSS nor a velocity in the catalog Rines *et al.* (2003).

We use the [Simple Spectral Access (SSA) protocol from the IVOA](http://www.ivoa.net/documents/SSA/) to query the Mikulski Archive for Space Telescopes (MAST). Once again, we look at an area of 40arcmin around the center of the Coma Cluster. 

In [ ]:
mast_ssa_service = pyvo.dal.SSAService(
    'https://archive.stsci.edu/ssap/search2.php?id=HST&')
diameter = u.Quantity(2 * 40.0, unit="arcmin")
position = SkyCoord.from_name('A1656')
mast_hst_results = mast_ssa_service.search(pos=position, diameter=diameter)
mast_hst_results


Note that `mast_hst_results` is not a list of tables as we had for `astroquery` queries. This time, we get a pyvo `resultset`. Thus the methods to handle the `resultset` are slightly different but can still be printed out with the `dir()` function which is generic in python. Let's find out which columns are available:

In [ ]:

print('--- Available methods: ', dir(mast_hst_results))
print('--- Name of columns: ', mast_hst_results.fieldnames)


In [ ]:
for observation in mast_hst_results:
    print(observation['object'])


Often Quasars are further away than the Coma cluster, so let's check quickly on Simbad whether this source is interesting for further analysis. Usually a Simbad query would only return information on the object's identifier and coordinates. We are, however, also interested in its redshift, so we first create a customised Simbad query (as we did above for VizieR, for more details see [here](https://astroquery.readthedocs.io/en/latest/simbad/simbad.html#customizing-the-default-settings)) and then submit the query.

In [ ]:
custom_Simbad = Simbad()
custom_Simbad.add_votable_fields('z_value')
qso_table = custom_Simbad.query_object('QSO 1258+285')
qso_table


As you can see in the last column, the Quasar is at a redshift of 1.36. This is far beyond the Coma Cluster. Therefore, we focus on the source '1257+2840' for now. 1257+2840 is the last source in the list: we assign it to a new variable (`interesting_observation`). Then we exploit the functionalities of `resultset` to find out where the data is and what kind of file it is. 

In [ ]:
interesting_observation = mast_hst_results[-1]
observation_url = interesting_observation.getdataurl()
print(observation_url)


### A quick analysis of the discovered spectrum

With the previous step, we obtained a link to a fits file which we can download and open with `astropy`.  

In [ ]:
spectrum_fits = fits.open(observation_url)
spectrum_fits.info()


In [ ]:
spectrum_fits[1].header


From the fits information and the header, it appears that we have three columns (listed in one axis though): 
- wavelength in Angstrom, 
- flux and flux error in $\mathrm{erg \cdot cm}^{-2} \mathrm{s}^{-1} \mathrm{\r{A}}^{-1}$.

For a first quick look we can plot the spectrum:

In [ ]:
fig = plt.figure(figsize=(10.0, 8.0))
ax = fig.add_axes([0.17, 0.17, 0.77, 0.77])
ax.plot(spectrum_fits[1].data[0][0], spectrum_fits[1].data[0][1])
ax.set_xlabel(r'Wavelength ($\mathrm{\AA}$)', fontsize=14)
ax.set_ylabel(r'Flux ($\mathrm{erg \cdot cm}^{-2} \mathrm{s}^{-1} \mathrm{\AA}^{-1}$)',
              fontsize=14)


It is a spectrum in the ultraviolet with two visible emission lines, one around $1220\mathrm{\r{A}}$ and one around $1330\mathrm{\r{A}}$. We know that wavelength at rest of the Lyman $\alpha$ line is at $1216\mathrm{\r{A}}$. This spectrum might thus show Ly$\alpha$ (atomic hydrogen, HI) emission of the Milky Way (hardly redshifted) and a redshifted extragalactic source. 

To investigate this further, we use the `specutils` package. First, we define a 1D spectrum: the data format that `specutils` accepts. 

In [ ]:
flux_unit = u.erg / u.cm**2 / u.s / u.Angstrom
spectrum = Spectrum1D(spectral_axis=spectrum_fits[1].data[0][0] * u.Angstrom,
                      flux=spectrum_fits[1].data[0][1] * flux_unit)
spectrum


Now we can use `specutils_analysis` functions to analyze the spectrum. Let's find the centroid of the two lines.

In [ ]:
# find the first peak, between 1200 and 1260 Angstrom
centroid_Milky_Way = spec_ana.centroid(spectrum, SpectralRegion(1200 * u.Angstrom,
                                                                1260 * u.Angstrom))
# find the second peak, between 1300 and 1370 Angstrom
centroid_second_peak = spec_ana.centroid(spectrum, SpectralRegion(1300 * u.Angstrom,
                                                                  1370 * u.Angstrom))
print('The centroid of the first peak is located at: ', centroid_Milky_Way)
print('The centroid of the second peak is located at: ', centroid_second_peak)


Indeed the first peak is centered around the rest wavelength of the Ly$\alpha$ line. We may thus assume that this is the HI emission from the Milky Way in the foreground. Now assuming that the second line is also Ly$\alpha$ emission, let's calculate the redshift and recession velocity:

In [ ]:
rest_Ly_alpha = 1216.
redshift_z = (centroid_second_peak.value - rest_Ly_alpha) / rest_Ly_alpha
cz = redshift_z * c  # speed of light in km/s, defined before
print(
    f'The source has a redshift of {round(redshift_z, 3)} and a recession velocity of {round(cz, 2)} km/s ')


Although this source is much closer than the Quasar, it is still further away than the Coma Cluster and thus not a member of the Cluster. 

An alternative to using `specutils` we can also use more generic python packages and fit the emission lines with simple Gaussians. Let's define the Gaussian:

In [5]:
def gauss(x, height, peak_value, sigma):
    """Gaussian 1d function

    Parameters
    ----------
    x : numpy 1d array or a list
    height : float
    peak_value : float
    sigma : float

    Returns
    -------
    numpy array
    """
    return height * np.exp(-(np.asarray(x) - peak_value)**2. / (2 * sigma**2.))


Next, we select the two parts of the spectrum where the emission lines are:

In [ ]:
# separate wavelengths and fluxes in different objects
spectrum_wavelengths = spectrum_fits[1].data[0][0]
spectrum_flux = spectrum_fits[1].data[0][1]

# make a mask to select wavelengths between 1190 and 1240 Angstrom
mask_Milky_Way = np.where((spectrum_wavelengths < 1240.)
                          & (spectrum_wavelengths > 1190))
wavelengths_Milky_Way = spectrum_wavelengths[mask_Milky_Way]
flux_Milky_Way = spectrum_flux[mask_Milky_Way]

# make a mask to select wavelengths between 1300 and 1380 Angstrom
mask_second_peak = np.where((spectrum_wavelengths < 1380)
                            & (spectrum_wavelengths > 1300))
wavelengths_second_peak = spectrum_wavelengths[mask_second_peak]
flux_second_peak = spectrum_flux[mask_second_peak]


The Gaussian fit is done with the `curve_fit` function of the `scipy.optimize` library.

In [ ]:
popt_Milky_Way_line, pcov_Milky_Way_line = curve_fit(gauss, wavelengths_Milky_Way,
                                                     flux_Milky_Way,
                                                     p0=[1.25e-13, 1220., 10.0])
print(f"""The first peak, attributed to the Milky Way, has a 
central wavelength of {round(popt_Milky_Way_line[1], 2)} +/- {round(np.sqrt(np.diag(pcov_Milky_Way_line))[1], 2)} km/s.""")


popt_second_line, pcov_second_line = curve_fit(gauss, wavelengths_second_peak,
                                               flux_second_peak,
                                               p0=[0.2e-13, 1330., 10.0])
print(f"""
The first peak, for the other object, has a 
central wavelength of {round(popt_second_line[1], 2)} +/- {round(np.sqrt(np.diag(pcov_second_line))[1], 2)} km/s.""")


These values are close to the results from the `specutils` package. 

To further check the fitting results, we plot the data, our model, and residuals:

In [ ]:
import wave


fig, axes = plt.subplots(2, 1)

# First plot is the data and models
sns.lineplot(x=wavelengths_Milky_Way,
             y=flux_Milky_Way, ax=axes[0])
sns.lineplot(x=wavelengths_Milky_Way,
             y=gauss(wavelengths_Milky_Way, *popt_Milky_Way_line), 
             ls='-', color='k', ax=axes[0])
sns.lineplot(x=wavelengths_second_peak,
             y=flux_second_peak, ax=axes[0])
sns.lineplot(x=wavelengths_second_peak,
             y=gauss(wavelengths_second_peak, *popt_second_line), 
             ls='-', color='k', ax=axes[0], label='gaussian fits')
axes[0].legend()
axes[0].set_ylabel('Flux [erg cm$^{-2}$ s$^{-1}$ $\AA^{-1}$]')


# Second plot is the residuals = data - model
sns.lineplot(x=wavelengths_Milky_Way,
             y=flux_Milky_Way - gauss(wavelengths_Milky_Way,
                                      *popt_Milky_Way_line),
             ax=axes[1]
             )
sns.lineplot(x=wavelengths_second_peak,
             y=flux_second_peak -
             gauss(wavelengths_second_peak, *popt_second_line),
             ax=axes[1])
axes[1].axhline(0, ls='--', c='k')
axes[1].set_xlabel(r'Wavelength [$\mathrm{\AA}$]')
axes[1].set_ylabel('Residual')


While fitting a Gaussian to the emission lines provides very good results with regard to the central wavelength (and thus redshift) of the observed objects, the residuals show that the emission from the Milky Way is much more complicated. 

In [ ]:
# End of the tutorial
spectrum_fits.close()
